<a href="https://colab.research.google.com/github/NishaMDev/DeepLearning/blob/main/Assignment%237/DL_Assignment_7_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 2: Demonstrate very simple many to one, one to many and many to many RNN colabs 

## Types of Sequence Problems
Sequence problems can be broadly categorized into the following categories:



*   One-to-One: Where there is one input and one output. Typical example of a 
one-to-one sequence problems is the case where you have an image and you want to predict a single label for the image.
*   Many-to-One: In many-to-one sequence problems, we have a sequence of data as input and we have to predict a single output. Text classification is a prime example of many-to-one sequence problems where we have an input sequence of words and we want to predict a single output tag.
*   One-to-Many: In one-to-many sequence problems, we have single input and a sequence of outputs. A typical example is an image and its corresponding description.
*   Many-to-Many: Many-to-many sequence problems involve a sequence input and a sequence output. For instance, stock prices of 7 days as input and stock prices of next 7 days as outputs. Chatbots are also an example of many-to-many sequence problems where a text sequence is an input and another text sequence is the output.









## RNN - One-to-one

> In this section, we will see how to solve one-to-one sequence problem where each time-step has a single feature.

> Let's first import the required libraries that we are going to use in this article:

In [1]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.layers import Bidirectional

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

**Creating the Dataset**

In this next step, we will prepare the dataset that we are going to use for this section.

In the script above, we create 20 inputs and 20 outputs. Each input consists of one time-step, which in turn contains a single feature. Each output value is 15 times the corresponding input value. If you run the above script, you should see the input and output values as shown below:

In [2]:
X = list()
Y = list()
X = [x+1 for x in range(20)]
Y = [y * 15 for y in X]

print(X)
print(Y)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
[15, 30, 45, 60, 75, 90, 105, 120, 135, 150, 165, 180, 195, 210, 225, 240, 255, 270, 285, 300]


The input to LSTM layer should be in 3D shape i.e. (samples, time-steps, features). The samples are the number of samples in the input data. We have 20 samples in the input. The time-steps is the number of time-steps per sample. We have 1 time-step. Finally, features correspond to the number of features per time-step. We have one feature per time-step.

We can reshape our data via the following command:

In [3]:
X = np.array(X).reshape(20, 1, 1)
Y = np.array(Y)

**Solution via Simple LSTM**

Now we can create our simple LSTM model with one LSTM layer.



In [4]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                10400     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________
None


In the script above, we create an LSTM model with one LSTM layer of 50 neurons and relu activation functions. You can see the input shape is (1,1) since our data has one time-step with one feature. Executing the above script prints the following summary:

Let's now train our model:

In [5]:
model.fit(X, Y, epochs=2000, validation_split=0.2, batch_size=5)

Epoch 1/2000
4/4 [==============================] - 2s 81ms/step - loss: 20965.9141 - val_loss: 76936.0703
Epoch 2/2000
4/4 [==============================] - 0s 11ms/step - loss: 20936.2617 - val_loss: 76832.5312
Epoch 3/2000
4/4 [==============================] - 0s 10ms/step - loss: 20909.2832 - val_loss: 76726.4688
Epoch 4/2000
4/4 [==============================] - 0s 11ms/step - loss: 20878.5840 - val_loss: 76617.9531
Epoch 5/2000
4/4 [==============================] - 0s 11ms/step - loss: 20847.9023 - val_loss: 76498.4531
Epoch 6/2000
4/4 [==============================] - 0s 11ms/step - loss: 20820.6758 - val_loss: 76368.8906
Epoch 7/2000
4/4 [==============================] - 0s 11ms/step - loss: 20783.3516 - val_loss: 76243.8828
Epoch 8/2000
4/4 [==============================] - 0s 12ms/step - loss: 20747.0742 - val_loss: 76116.3125
Epoch 9/2000
4/4 [==============================] - 0s 11ms/step - loss: 20713.2969 - val_loss: 75972.1719
Epoch 10/2000
4/4 [==================

We train our model for 2000 epochs with a batch size of 5. You can choose any number. Once the model is trained, we can make predictions on a new instance.

Let's say we want to predict the output for an input of 30. The actual output should be 30 x 15 = 450. Let's see what value do we get. First, we need to convert our test data to the right shape i.e. 3D shape, as expected by LSTM. The following script predicts the output for the number 30:

In [6]:
test_input = array([30])
test_input = test_input.reshape((1, 1, 1))
test_output = model.predict(test_input, verbose=0)
print(test_output)

[[440.95642]]


## RNN - One-to-many

> One-to-many sequence problems are the type of sequence problems where input data has one time-step and the output contains a vector of multiple values or multiple time-steps. 

> In this section, we will see how to solve one-to-many sequence problems where the input has a single feature. We will then move on to see how to work with multiple features input to solve one-to-many sequence problems.



**Creating the Dataset**

In [7]:
X = list()
Y = list()
X = [x+3 for x in range(-2, 43, 3)]

for i in X:
    output_vector = list()
    output_vector.append(i+1)
    output_vector.append(i+2)
    Y.append(output_vector)

print(X)
print(Y)

[1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43]
[[2, 3], [5, 6], [8, 9], [11, 12], [14, 15], [17, 18], [20, 21], [23, 24], [26, 27], [29, 30], [32, 33], [35, 36], [38, 39], [41, 42], [44, 45]]


Our input contains 15 samples with one time-step and one feature value. For each value in the input sample, the corresponding output vector contains the next two integers. For instance, if the input is 4, the output vector will contain values 5 and 6. Hence, the problem is a simple one-to-many sequence problem.

The following script reshapes our data as required by the LSTM:

In [8]:
X = np.array(X).reshape(15, 1, 1)
Y = np.array(Y)

Solution via Simple LSTM

In [9]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, 1)))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mse')
model.fit(X, Y, epochs=1000, validation_split=0.2, batch_size=3)


Epoch 1/1000
4/4 [==============================] - 1s 63ms/step - loss: 456.8681 - val_loss: 1670.0317
Epoch 2/1000
4/4 [==============================] - 0s 8ms/step - loss: 449.1221 - val_loss: 1641.0839
Epoch 3/1000
4/4 [==============================] - 0s 8ms/step - loss: 440.1195 - val_loss: 1611.3224
Epoch 4/1000
4/4 [==============================] - 0s 8ms/step - loss: 431.4444 - val_loss: 1580.5903
Epoch 5/1000
4/4 [==============================] - 0s 8ms/step - loss: 422.5240 - val_loss: 1548.5641
Epoch 6/1000
4/4 [==============================] - 0s 8ms/step - loss: 413.1139 - val_loss: 1513.5951
Epoch 7/1000
4/4 [==============================] - 0s 8ms/step - loss: 403.9318 - val_loss: 1473.7480
Epoch 8/1000
4/4 [==============================] - 0s 8ms/step - loss: 393.1743 - val_loss: 1429.5033
Epoch 9/1000
4/4 [==============================] - 0s 9ms/step - loss: 380.7390 - val_loss: 1380.0928
Epoch 10/1000
4/4 [==============================] - 0s 8ms/step - loss:

Once the model is trained we can make predictions on the test data:

In [10]:
test_input = array([10])
test_input = test_input.reshape((1, 1, 1))
test_output = model.predict(test_input, verbose=0)
print(test_output)

[[11.047798 12.01309 ]]


The test data contains a value 10. In the output, we should get a vector containing 11 and 12. The output I received is [11.047798 12.01309 ] which is actually very close to the expected output.

## RNN - Many-to-one
> In this post, We will briefly cover the many-to-one type, which is one the common types of Recurrent Neural Network and its implementation in tensorflow. 


Let's first create the dataset. Our dataset will consist of 15 samples. Each sample will have 3 time-steps where each time-step will consist of a single feature i.e. a number. The output for each sample will be the sum of the numbers in each of the three time-steps. For instance, if our sample contains a sequence 4,5,6 the output will be 4 + 5 + 6 = 10.

**Creating the Dataset**

Let's first create a list of integers from 1 to 45. Since we want 15 samples in our dataset, we will reshape the list of integers containing the first 45 integers.

In [11]:
X = np.array([x+1 for x in range(45)])
print(X)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45]


We can reshape it into number of samples, time-steps and features using the following function:

In [12]:
X = X.reshape(15,3,1)
print(X)

[[[ 1]
  [ 2]
  [ 3]]

 [[ 4]
  [ 5]
  [ 6]]

 [[ 7]
  [ 8]
  [ 9]]

 [[10]
  [11]
  [12]]

 [[13]
  [14]
  [15]]

 [[16]
  [17]
  [18]]

 [[19]
  [20]
  [21]]

 [[22]
  [23]
  [24]]

 [[25]
  [26]
  [27]]

 [[28]
  [29]
  [30]]

 [[31]
  [32]
  [33]]

 [[34]
  [35]
  [36]]

 [[37]
  [38]
  [39]]

 [[40]
  [41]
  [42]]

 [[43]
  [44]
  [45]]]


The above script converts the list X into 3-dimensional shape with 15 samples, 3 time-steps, and 1 feature. The script above also prints the reshaped data.



We have converted our input data into the right format, let's now create our output vector. As I said earlier, each element in the output will be equal to the sum of the values in the time-steps in the corresponding input sample. The following script creates the output vector:

In [13]:
Y = list()
for x in X:
    Y.append(x.sum())

Y = np.array(Y)
print(Y)

[  6  15  24  33  42  51  60  69  78  87  96 105 114 123 132]


Let's now create our model with one LSTM layer.

In [14]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(3, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

The following script trains our model:

In [15]:
history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1)

Epoch 1/1000
1/1 [==============================] - 1s 1s/step - loss: 3925.8857 - val_loss: 14682.2500
Epoch 2/1000
1/1 [==============================] - 0s 26ms/step - loss: 3903.5256 - val_loss: 14593.2979
Epoch 3/1000
1/1 [==============================] - 0s 26ms/step - loss: 3880.3772 - val_loss: 14500.5771
Epoch 4/1000
1/1 [==============================] - 0s 26ms/step - loss: 3856.3489 - val_loss: 14403.5811
Epoch 5/1000
1/1 [==============================] - 0s 25ms/step - loss: 3831.2922 - val_loss: 14302.3408
Epoch 6/1000
1/1 [==============================] - 0s 26ms/step - loss: 3805.2773 - val_loss: 14196.9541
Epoch 7/1000
1/1 [==============================] - 0s 25ms/step - loss: 3778.2285 - val_loss: 14086.1162
Epoch 8/1000
1/1 [==============================] - 0s 27ms/step - loss: 3750.0078 - val_loss: 13969.1377
Epoch 9/1000
1/1 [==============================] - 0s 26ms/step - loss: 3720.4812 - val_loss: 13845.3740
Epoch 10/1000
1/1 [=============================

Once the model is trained, we can use it to make predictions on the test data points. Let's predict the output for the number sequence 50,51,52. The actual output should be 50 + 51 + 52 = 153. The following script converts our test points into a 3-dimensional shape and then predicts the output:

In [16]:
test_input = array([50,51,52])
test_input = test_input.reshape((1, 3, 1))
test_output = model.predict(test_input, verbose=0)
print(test_output)

[[154.74905]]


I got 153.53 in the output, which is around the actual output value of 153.

## RNN - Many-to-many


> In this section we will solve many-to-many sequence problems via the encoder-decoder model, where each time-step in the input sample will contain one feature.

Let's first create our dataset.

**Creating the Dataset**

In [17]:
X = list()
Y = list()
X = [x for x in range(5, 301, 5)]
Y = [y for y in range(20, 316, 5)]

X = np.array(X).reshape(20, 3, 1)
Y = np.array(Y).reshape(20, 3, 1)

The input X contains 20 samples where each sample contains 3 time-steps with one feature. One input sample looks like this:

You can see that the input sample contain 3 values that are basically 3 consecutive multiples of 5. The corresponding output sequence for the above input sample is as follows:

The output contains the next three consecutive multiples of 5. You can see the output in this case is different than what we have seen in the previous sections. For the encoder-decoder model, the output should also be converted into a 3D format containing the number of samples, time-steps, and features. The is because the decoder generates an output per time-step.

We have created our dataset; the next step is to train our models. We will train stacked LSTM and bidirectional LSTM models in the following sections.



The following script creates the encoder-decoder model using stacked LSTMs:

In [18]:
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

model = Sequential()

# encoder layer
model.add(LSTM(100, activation='relu', input_shape=(3, 1)))

# repeat vector
model.add(RepeatVector(3))

# decoder layer
model.add(LSTM(100, activation='relu', return_sequences=True))

model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')


In the above script, the first LSTM layer is the encoder layer.

Next, we have added the repeat vector to our model. The repeat vector takes the output from encoder and feeds it repeatedly as input at each time-step to the decoder. For instance, in the output we have three time-steps. To predict each output time-step, the decoder will use the value from the repeat vector, the hidden state from the previous output and the current input.

Next we have a decoder layer. Since the output is in the form of a time-step, which is a 3D format, the return_sequences for the decoder model has been set True. The TimeDistributed layer is used to individually predict the output for each time-step.

The model summary for the encoder-decoder model created in the script above is as follows:

In [19]:
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 100)               40800     
                                                                 
 repeat_vector (RepeatVector  (None, 3, 100)           0         
 )                                                               
                                                                 
 lstm_4 (LSTM)               (None, 3, 100)            80400     
                                                                 
 time_distributed (TimeDistr  (None, 3, 1)             101       
 ibuted)                                                         
                                                                 
Total params: 121,301
Trainable params: 121,301
Non-trainable params: 0
_________________________________________________________________
None


You can see that the repeat vector only repeats the encoder output and has no parameters to train.

The following script trains the above encoder-decoder model.

In [20]:
history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3)

Epoch 1/1000
6/6 [==============================] - 2s 70ms/step - loss: 21488.3652 - val_loss: 64723.8750
Epoch 2/1000
6/6 [==============================] - 0s 16ms/step - loss: 17237.5371 - val_loss: 44315.8789
Epoch 3/1000
6/6 [==============================] - 0s 15ms/step - loss: 10257.2344 - val_loss: 19847.2578
Epoch 4/1000
6/6 [==============================] - 0s 15ms/step - loss: 5597.4795 - val_loss: 20788.2500
Epoch 5/1000
6/6 [==============================] - 0s 16ms/step - loss: 4926.0698 - val_loss: 14704.9541
Epoch 6/1000
6/6 [==============================] - 0s 16ms/step - loss: 3929.6052 - val_loss: 12458.7012
Epoch 7/1000
6/6 [==============================] - 0s 16ms/step - loss: 3234.8311 - val_loss: 9074.4355
Epoch 8/1000
6/6 [==============================] - 0s 15ms/step - loss: 2229.1958 - val_loss: 5935.9600
Epoch 9/1000
6/6 [==============================] - 0s 16ms/step - loss: 1205.7302 - val_loss: 2999.1035
Epoch 10/1000
6/6 [===========================

Let's create a test-point and see if our encoder-decoder model is able to predict the multi-step output. Execute the following script:

In [21]:
test_input = array([300, 305, 310])
test_input = test_input.reshape((1, 3, 1))
test_output = model.predict(test_input, verbose=0)

Our input sequence contains three time-step values 300, 305 and 310. The output should be next three multiples of 5 i.e. 315, 320 and 325. I received the following output

In [22]:
print(test_output)

[[[313.4486 ]
  [318.3733 ]
  [323.06985]]]


You can see that the output is in 3D format.